# Test reading csv and excel

In [ ]:
#r "nuget: FSharp.Data"
open System
open System.Data
open FSharp.Data

Installed Packages FSharp.Data, 4.2.6

## read a csv file into a DataSet

In [ ]:
let dir = "/Users/mmgreiner/Projects/OperationResearch/ProductFinderCS"

let enumFiles dir pat =
    IO.Directory.EnumerateFiles(dir, "*.csv") |> Seq.toList

let files = enumFiles dir "*.csv"
files


index,value
0,/Users/mmgreiner/Projects/OperationResearch/ProductFinderCS/100_DC_Motors_Parts - export from IFS.csv
1,/Users/mmgreiner/Projects/OperationResearch/ProductFinderCS/EngPart_characteristics - 100DC_Motors.csv


Now create a class that reads a csv file and turns the rows into '''DataRows'''


In [ ]:
let (|IsInt|_|) (s: string) = match Int32.TryParse s with | true, i -> Some i | _ -> None
let (|IsFloat|_|) (s: string) = match Double.TryParse s with | true, i -> Some i | _ -> None
let (|IsDateTime|_|) (s: string) = match Double.TryParse s with | true, i -> Some i | _ -> None
let (|IsBool|_|) (s: string) = match Boolean.TryParse s with | true, i -> Some i | _ -> None

let guessType (sl: string seq) =
    let mutable maybeOpt = false
    let mutable guessTyp = Unchecked.defaultof<Type>
    
    let guesser = function 
                    | t when String.IsNullOrEmpty t -> 
                        maybeOpt <- true
                    | IsFloat f -> guessTyp <- typedefof<float>
                    | IsInt f -> guessTyp <- typedefof<int>
                    | IsDateTime f -> guessTyp <- typedefof<DateTime>
                    | IsBool f -> guessTyp <- typedefof<bool>
                    | _ -> guessTyp <- typedefof<string>

    sl |> Seq.iter guesser
    


In [ ]:
type FSharp.Data.CsvFile with
    member this.toDataTable name = 
        let headers = this.Headers.Value
        let table = new DataTable(name)
        for h in headers do
            table.Columns.Add(new DataColumn(h))
        this.Rows |> Seq.iter (fun r -> 
            let dr = table.NewRow()
            let cols = [ for c in table.Columns do yield c ]
            cols |> Seq.iteri (fun i c ->
                dr.[i] <- r.[i]
            )
            table.Rows.Add(dr)
        )
        table

now try it

In [ ]:
let csv = CsvFile.Load(files.Head, separators = ";")
let table = csv.toDataTable "Charactistics"


In [ ]:
for c in table.Columns do
    printfn "col = %A" c

let r0 = table.Rows.[0].[2]
printfn "r0 = %A" r0

col = Column1
col = CONTRACT
col = PART_NO
col = IP_DESC
col = ASSET_CLASS
col = PART_STATUS
col = STD_NAME_ID
col = STD_NAME
col = ENG_REVISION
col = SUM(IPIS.QTY_ONHAND)
col = SUM(IPIS.QTY_RESERVED)
r0 = "110043"


Now try to guess the type

In [ ]:
let c0 = table.Columns.["PART_NO"]
printfn "colum %s = %A" c0.ColumnName table.Columns.[c0.ColumnName] 

//let c0 = table.DefaultView.ToTable(false, "CONTRACT")
printfn "r0 = %A" (tables.Rows.[0].[0])

colum PART_NO = PART_NO
r0 = "1;MCH;110043;A-max16 EBCLL 2W SL 1WE;FG;A;161;DC-Motor;05;259;0"


Now try how column names are transformed

In [ ]:
let tab2 = new DataTable("Test Column Names")
[ "Hallo Markkus"; "Hallo % Prozent"; "Hallo (%)"]
|> List.iter (fun h -> tab2.Columns.Add(new DataColumn(h)))

for c in tab2.Columns do
    printfn "name: %s/%s" c.ColumnName c.Caption


name: Hallo Markkus/Hallo Markkus
name: Hallo % Prozent/Hallo % Prozent
name: Hallo (%)/Hallo (%)
